In [1]:
from __future__ import print_function
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

ImportError: dlopen(/Users/Devansh/anaconda3/lib/python3.6/site-packages/torch/_C.cpython-36m-darwin.so, 9): Library not loaded: @rpath/libcaffe2.dylib
  Referenced from: /Users/Devansh/anaconda3/lib/python3.6/site-packages/torch/lib/libshm.dylib
  Reason: image not found